# 06.1 - Introduction to transformation processes

In the Silver layer of the lakehouse, the data from the Bronze layer is matched, merged, conformed and cleansed ("just-enough") so that the Silver layer can provide an "Enterprise view" of all its key business entities, concepts and transactions. (e.g. master customers, stores, non-duplicated transactions and cross-reference tables).

The Silver layer brings the data from different sources into an Enterprise view and enables self-service analytics for ad-hoc reporting, advanced analytics and ML. It serves as a source for Departmental Analysts, Data Engineers and Data Scientists to further create projects and analysis to answer business problems via enterprise and departmental data projects in the Gold Layer.

In the lakehouse data engineering paradigm, typically the ELT methodology is followed vs. ETL - which means only minimal or "just-enough" transformations and data cleansing rules are applied while loading the Silver layer. Speed and agility to ingest and deliver the data in the data lake is prioritized, and a lot of project-specific complex transformations and business rules are applied while loading the data from the Silver to Gold layer. From a data modeling perspective, the Silver Layer has more 3rd-Normal Form like data models. Data Vault-like, write-performant data models can be used in this layer.

## 1. Dbt and transformation engines


At the most basic level, dbt has two components: a compiler and a runner. Users write dbt code in their text editor of choice and then invoke dbt from the command line. dbt compiles all code into raw SQL and executes that code against the configured data warehouse. This allows us to swap the different transformation engines (i.e. Spark, Trino, PostgreSQL) without having to migrate the pipelines code to the new engine.

Each dbt project is organized into different folders like:


| Folder         | Purpose                                                                                   |
| -------------- | ----------------------------------------------------------------------------------------- |
| **models/**    | The heart of dbt: all your `.sql` models. Each becomes a table or view in your warehouse. |
| **macros/**    | Reusable SQL logic using Jinja — for surrogate keys, pivoting, dynamic filters, etc.      |
| **seeds/**     | CSVs that dbt loads into the database as tables. Useful for reference data.               |
| **snapshots/** | Handle slowly changing dimensions (track history of changes).                             |
| **tests/**     | Schema and data tests — either YAML-based or custom SQL.                                  |
| **analyses/**  | Saved queries or analysis code not materialized as models.                                |
| **target/**    | Auto-generated during compilation/runs — contains compiled SQL and logs.                  |


### 1.1 Using dbt to transform the data from one layer to another one

In the `project/dbt/silver` directory you can find the dbt project that contains all the logic to transform the bronze layer data into the silver layer data.


In the `models` folder you can found two important files:

- `source.yml`: this contains the description of all the bronze layer tables, including basic data integrity checks
- `schema.yml`: this contains the description of all the silver layer tables, including basic data integrity checks

Then for each one of the tables defined in the `schema.yml` file there should be a `<table_name>.sql` file with the dbt code to produce the given table

For this task you should 

1. Define all the bronze tables in the `source.yml` file with all integrity constraints that may apply
2. Define all the silver tables in the `schema.yml` file with all integrity constrainst that may apply
3. For each table in the silver layer create a `.sql` file to create the table

Once you have all the tables defined you can compile the dbt project using the `dbt compile` command at the projects location (`/var/lib/adventureworks/dbt/silver`)

In [5]:
!cd /var/lib/adventureworks/dbt/silver && dbt clean # always run this

22:54:26  Running with dbt=1.10.13
22:54:26  Checking /var/lib/adventureworks/dbt/silver/target/*
22:54:26  Cleaned /var/lib/adventureworks/dbt/silver/target/*
22:54:26  Finished cleaning all paths.


In [2]:
!cd /var/lib/adventureworks/dbt/silver && dbt deps # you can run this only one time to get the dependencies

06:38:45  Running with dbt=1.10.13
06:38:48  Installing dbt-labs/dbt_utils
06:38:50  Installed from version 1.3.2
06:38:50  Updated version available: 1.3.3
06:38:50  
06:38:50  Updates available for packages: ['dbt-labs/dbt_utils']                 
Update your versions in packages.yml, then run dbt deps


In [2]:
!cd /var/lib/adventureworks/dbt/silver && dbt compile

06:20:32  Running with dbt=1.10.13
INFO:trino.auth:keyring module not found. OAuth2 token will not be stored in keyring.
INFO:trino.auth:keyring module not found. OAuth2 token will not be stored in keyring.
06:20:32  Registered adapter: trino=1.9.3
06:20:34  Found 63 models, 879 data tests, 63 sources, 571 macros
06:20:34  
06:20:34  Concurrency: 1 threads (target='dev')
06:20:34  
06:20:34  [WARNING]: SSL certificate validation is disabled by default. It is legacy behavior which will be changed in future releases. It is strongly advised to enable `require_certificate_validation` flag or explicitly set `cert` configuration to `True` for security reasons. You may receive an error after that if your SSL setup is incorrect.
You may opt into the new behavior sooner by setting `flags.require_certificate_validation` to `True` in `dbt_project.yml`.
Visit https://docs.getdbt.com/reference/global-configs/behavior-changes for more information.


In [2]:
!cd /var/lib/adventureworks/dbt/gold && dbt clean # always run this
# !cd /var/lib/adventureworks/dbt/gold && dbt deps # you can run this only one time to get the dependencies
!cd /var/lib/adventureworks/dbt/gold && dbt compile

20:39:33  Running with dbt=1.10.13
20:39:33  Checking /var/lib/adventureworks/dbt/gold/target/*
20:39:33  Cleaned /var/lib/adventureworks/dbt/gold/target/*
20:39:33  Finished cleaning all paths.
20:39:39  Running with dbt=1.10.13
INFO:trino.auth:keyring module not found. OAuth2 token will not be stored in keyring.
INFO:trino.auth:keyring module not found. OAuth2 token will not be stored in keyring.
20:39:40  Registered adapter: trino=1.9.3
20:39:40  Unable to do partial parsing because saved manifest not found. Starting full parse.
20:39:42  [WARNING]: Did not find matching node for patch with name 'dimproduct1' in the 'models' section of file 'models/schema.yml'
20:39:42  [WARNING]: Did not find matching node for patch with name 'factsale' in the 'models' section of file 'models/schema.yml'
20:39:43  [WARNING]: Test 'test.gold.not_null_dimproduct1_productid.ffb707ad6a' (models/schema.yml) depends on a node named 'dimproduct1' in package '' which was not found
20:39:43  [WARNING]: Test

In [6]:
!cd /var/lib/adventureworks/dbt/gold && dbt clean # always run this

21:09:40  Running with dbt=1.10.13
21:09:40  Checking /var/lib/adventureworks/dbt/gold/target/*
21:09:40  Cleaned /var/lib/adventureworks/dbt/gold/target/*
21:09:40  Finished cleaning all paths.


In [16]:
!cd /var/lib/adventureworks/dbt/gold && dbt compile

21:54:01  Running with dbt=1.10.13
INFO:trino.auth:keyring module not found. OAuth2 token will not be stored in keyring.
INFO:trino.auth:keyring module not found. OAuth2 token will not be stored in keyring.
21:54:02  Registered adapter: trino=1.9.3
21:54:03  Found 11 models, 38 data tests, 21 sources, 571 macros
21:54:03  
21:54:03  Concurrency: 1 threads (target='dev')
21:54:03  
21:54:03  [WARNING]: SSL certificate validation is disabled by default. It is legacy behavior which will be changed in future releases. It is strongly advised to enable `require_certificate_validation` flag or explicitly set `cert` configuration to `True` for security reasons. You may receive an error after that if your SSL setup is incorrect.
You may opt into the new behavior sooner by setting `flags.require_certificate_validation` to `True` in `dbt_project.yml`.
Visit https://docs.getdbt.com/reference/global-configs/behavior-changes for more information.


## Running dbt with Dagster

To run dbt with Dagster we create an asset in the `assets/silver` folder that runs the dbt CLI commands to build the tables. This
resources come from a `dagster-dbt` package that provides dbt support for Dagster.

To materialize the dbt assets in the project you can use the `silver_assets_job`